Write a function GreedyMotifSearchWithPseudocounts(Dna, k, t) that takes a list of strings Dna followed by integers k and t and returns the result of running GreedyMotifSearch, where each profile matrix is generated with pseudocounts. Then add this function to Motifs.py. (Hint: Ideally, you should only need an extremely small modification to your original GreedyMotifSearch function.)

In [1]:
# Input:  String Text and profile matrix Profile
# Output: Pr(Text, Profile)
def Pr(Motif, Profile):
    prob=1.0
    for char in range(len(Motif)):
        prob *= Profile[Motif[char]][char] # What is the probability that this character is in this position?
    
    return prob

def Score(Motifs):
    #This really seems inefficient. We could have consensus return both values...
    counts = CountWithPseudocounts(Motifs)
    consensus, counts = Consensus(Motifs)
    score = 0
    for char in range(len(consensus)):
        nucleotide = consensus[char] #Our nucleoutide
        keys = [key for key in ['A','C',"G","T"] if key != nucleotide ] # What are the non-consensus nucleotides
        for key in keys:# For each key
            score += counts[key][char] # add the number of times we were incorrect to our score
    return score

# Input:  A set of kmers Motifs
# Output: A consensus string of Motifs.
def Consensus(Motifs):
    consensus = "" #empty string.
    counts = CountWithPseudocounts(Motifs)

    for j in range(len(Motifs[0])):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if counts[symbol][j] > m:
                m = counts[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol # Add most frequent symbol
    return consensus, counts

# Input:  A set of kmers Motifs
# Output: CountWithPseudocounts(Motifs)
def CountWithPseudocounts(Motifs):
    count = {} # initializing the count dictionary
    #Initialize each nucleotide with an empty list, 
    for nucleotide in ["A","C","G","T"]:
        count[nucleotide] = []     
    for ind in range(len(Motifs[0])):
        for nucleotide in ["A","C","G","T"]:
            count[nucleotide].append(1.0) #everything must have a 1 initially  (Now that we are dealing with pseudo counts)
        for motif in range(len(Motifs)): #For each Motif, loop through chars
            count[Motifs[motif][ind]][ind] += 1.0 # FOr each nuc, increment its count for that Motif
    return count

# Write your ProfileMostProbableKmer() function here.
# The profile matrix assumes that the first row corresponds to A, the second corresponds to C,
# the third corresponds to G, and the fourth corresponds to T.
# You should represent the profile matrix as a dictionary whose keys are 'A', 'C', 'G', and 'T' and whose values are lists of floats
def ProfileMostProbableKmer(text, k, Profile):
    most_prob = ""
    high_prob = -1.0
    for index in range(len(text)-k+1):
        Motif = text[index:index+k]
        Motif_prob = Pr(Motif, Profile)
        if Motif_prob > high_prob:
            high_prob = Motif_prob
            most_prob = Motif
    return most_prob

# Input:  A set of kmers Motifs
# Output: ProfileWithPseudocounts(Motifs)
def ProfileWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    profile = {}
    counts = CountWithPseudocounts(Motifs)

    for nucleotide in ["A","C","G","T"]:
        #everything must divided by the total number of Motifs  
        #We added a single count to each nucleotide, hence the +4.0
        profile[nucleotide] = [ float(count)/(float(t) + 4.0) for count in counts[nucleotide]]  
    return profile

def GreedyMotifSearchWithPseudocounts(Dna, k, t):
    BestMotifs = []
    for i in range(0, t):
        BestMotifs.append(Dna[i][0:k])
        
    n = len(Dna[0])
    for i in range(n-k+1):
        Motifs = []
        Motifs.append(Dna[0][i:i+k])
        for j in range(1, t):
            P = ProfileWithPseudocounts(Motifs[0:j])
            Motifs.append(ProfileMostProbableKmer(Dna[j], k, P))
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs
    return BestMotifs

In [14]:
"""Test 0 # Sample Dataset (your code is not run on this dataset)
Input:"""

"""    3 5
    GGCGTTCAGGCA
    AAGAATCAGTCA
    CAAGGAGTTCGC
    CACGTCAATCAC
    CAATAATATTCG
Output:
    TTC
    ATC
    TTC
    ATC
    TTC

Test 1 # Check if you're missing the first kmer in each string of Dna
Input
    5 8
    AGGCGGCACATCATTATCGATAACGATTCGCCGCATTGCC
    ATCCGTCATCGAATAACTGACACCTGCTCTGGCACCGCTC
    AAGCGTCGGCGGTATAGCCAGATAGTGCCAATAATTTCCT
    AGTCGGTGGTGAAGTGTGGGTTATGGGGAAAGGCAGACTG
    AACCGGACGGCAACTACGGTTACAACGCAGCAAGAATATT
    AGGCGTCTGTTGTTGCTAACACCGTTAAGCGACGGCAACT
    AAGCGGCCAACGTAGGCGCGGCTTGGCATCTCGGTGTGTG
    AATTGAAAGGCGCATCTTACTCTTTTCGCTTTCAAAAAAA
Output:
    AGGCG
    ATCCG
    AAGCG
    AGTCG
    AACCG
    AGGCG
    AGGCG
    AGGCG

Test 2 # Check if you're missing the last kmer in each string of Dna
Input
    5 8
    GCACATCATTAAACGATTCGCCGCATTGCCTCGATAGGCG
    TCATAACTGACACCTGCTCTGGCACCGCTCATCCGTCGAA
    AAGCGGGTATAGCCAGATAGTGCCAATAATTTCCTTCGGC
    AGTCGGTGGTGAAGTGTGGGTTATGGGGAAAGGCAGACTG
    AACCGGACGGCAACTACGGTTACAACGCAGCAAGAATATT
    AGGCGTCTGTTGTTGCTAACACCGTTAAGCGACGGCAACT
    AAGCTTCCAACATCGTCTTGGCATCTCGGTGTGTGAGGCG
    AATTGAACATCTTACTCTTTTCGCTTTCAAAAAAAAGGCG
Output:
    AGGCG
    TGGCA
    AAGCG
    AGGCA
    CGGCA
    AGGCG
    AGGCG
    AGGCG

Test 3 # Check if you're not breaking ties correctly (you should pick the first kmer in the case of a tie)
Input
    5 8
    GCACATCATTATCGATAACGATTCATTGCCAGGCGGCCGC
    TCATCGAATAACTGACACCTGCTCTGGCTCATCCGACCGC
    TCGGCGGTATAGCCAGATAGTGCCAATAATTTCCTAAGCG
    GTGGTGAAGTGTGGGTTATGGGGAAAGGCAGACTGAGTCG
    GACGGCAACTACGGTTACAACGCAGCAAGAATATTAACCG
    TCTGTTGTTGCTAACACCGTTAAGCGACGGCAACTAGGCG
    GCCAACGTAGGCGCGGCTTGGCATCTCGGTGTGTGAAGCG
    AAAGGCGCATCTTACTCTTTTCGCTTTCAAAAAAAAATTG
Output:
    GGCGG
    GGCTC
    GGCGG
    GGCAG
    GACGG
    GACGG
    GGCGC
    GGCGC

Test 4 # Full dataset
Input:"""
k,t =3, 8
Dna4= ['GCACATCATTATCGATAACGATTCATTGCCAGGCGGCCGC',
    'TCATCGAATAACTGACACCTGCTCTGGCTCATCCGACCGC',
    'TCGGCGGTATAGCCAGATAGTGCCAATAATTTCCTAAGCG',
    'GTGGTGAAGTGTGGGTTATGGGGAAAGGCAGACTGAGTCG',
    'GACGGCAACTACGGTTACAACGCAGCAAGAATATTAACCG',
    'TCTGTTGTTGCTAACACCGTTAAGCGACGGCAACTAGGCG',
    'GCCAACGTAGGCGCGGCTTGGCATCTCGGTGTGTGAAGCG',
    'AAAGGCGCATCTTACTCTTTTCGCTTTCAAAAAAAAATTG']
#Output:
out4=    ["GGC",
    "GGC",
    "GGC",
    "GGC",
    "GGC",
    "GGC",
    "GGC",
    "GGC"]
assert (GreedyMotifSearch(Dna4, k, t) == out4)

In [15]:
GreedyMotifSearch(Dna4, k, t)

['GGC', 'GGC', 'GGC', 'GGC', 'GGC', 'GGC', 'GGC', 'GGC']

In [18]:


Dna = ["GCGCCCCGCCCGGACAGCCATGCGCTAACCCTGGCTTCGATGGCGCCGGCTCAGTTAGGGCCGGAAGTCCCCAATGTGGCAGACCTTTCGCCCCTGGCGGACGAATGACCCCAGTGGCCGGGACTTCAGGCCCTATCGGAGGGCTCCGGCGCGGTGGTCGGATTTGTCTGTGGAGGTTACACCCCAATCGCAAGGATGCATTATGACCAGCGAGCTGAGCCTGGTCGCCACTGGAAAGGGGAGCAACATC",
"CCGATCGGCATCACTATCGGTCCTGCGGCCGCCCATAGCGCTATATCCGGCTGGTGAAATCAATTGACAACCTTCGACTTTGAGGTGGCCTACGGCGAGGACAAGCCAGGCAAGCCAGCTGCCTCAACGCGCGCCAGTACGGGTCCATCGACCCGCGGCCCACGGGTCAAACGACCCTAGTGTTCGCTACGACGTGGTCGTACCTTCGGCAGCAGATCAGCAATAGCACCCCGACTCGAGGAGGATCCCG",
"ACCGTCGATGTGCCCGGTCGCGCCGCGTCCACCTCGGTCATCGACCCCACGATGAGGACGCCATCGGCCGCGACCAAGCCCCGTGAAACTCTGACGGCGTGCTGGCCGGGCTGCGGCACCTGATCACCTTAGGGCACTTGGGCCACCACAACGGGCCGCCGGTCTCGACAGTGGCCACCACCACACAGGTGACTTCCGGCGGGACGTAAGTCCCTAACGCGTCGTTCCGCACGCGGTTAGCTTTGCTGCC",
"GGGTCAGGTATATTTATCGCACACTTGGGCACATGACACACAAGCGCCAGAATCCCGGACCGAACCGAGCACCGTGGGTGGGCAGCCTCCATACAGCGATGACCTGATCGATCATCGGCCAGGGCGCCGGGCTTCCAACCGTGGCCGTCTCAGTACCCAGCCTCATTGACCCTTCGACGCATCCACTGCGCGTAAGTCGGCTCAACCCTTTCAAACCGCTGGATTACCGACCGCAGAAAGGGGGCAGGAC",
"GTAGGTCAAACCGGGTGTACATACCCGCTCAATCGCCCAGCACTTCGGGCAGATCACCGGGTTTCCCCGGTATCACCAATACTGCCACCAAACACAGCAGGCGGGAAGGGGCGAAAGTCCCTTATCCGACAATAAAACTTCGCTTGTTCGACGCCCGGTTCACCCGATATGCACGGCGCCCAGCCATTCGTGACCGACGTCCCCAGCCCCAAGGCCGAACGACCCTAGGAGCCACGAGCAATTCACAGCG",
"CCGCTGGCGACGCTGTTCGCCGGCAGCGTGCGTGACGACTTCGAGCTGCCCGACTACACCTGGTGACCACCGCCGACGGGCACCTCTCCGCCAGGTAGGCACGGTTTGTCGCCGGCAATGTGACCTTTGGGCGCGGTCTTGAGGACCTTCGGCCCCACCCACGAGGCCGCCGCCGGCCGATCGTATGACGTGCAATGTACGCCATAGGGTGCGTGTTACGGCGATTACCTGAAGGCGGCGGTGGTCCGGA",
"GGCCAACTGCACCGCGCTCTTGATGACATCGGTGGTCACCATGGTGTCCGGCATGATCAACCTCCGCTGTTCGATATCACCCCGATCTTTCTGAACGGCGGTTGGCAGACAACAGGGTCAATGGTCCCCAAGTGGATCACCGACGGGCGCGGACAAATGGCCCGCGCTTCGGGGACTTCTGTCCCTAGCCCTGGCCACGATGGGCTGGTCGGATCAAAGGCATCCGTTTCCATCGATTAGGAGGCATCAA",
"GTACATGTCCAGAGCGAGCCTCAGCTTCTGCGCAGCGACGGAAACTGCCACACTCAAAGCCTACTGGGCGCACGTGTGGCAACGAGTCGATCCACACGAAATGCCGCCGTTGGGCCGCGGACTAGCCGAATTTTCCGGGTGGTGACACAGCCCACATTTGGCATGGGACTTTCGGCCCTGTCCGCGTCCGTGTCGGCCAGACAAGCTTTGGGCATTGGCCACAATCGGGCCACAATCGAAAGCCGAGCAG",
"GGCAGCTGTCGGCAACTGTAAGCCATTTCTGGGACTTTGCTGTGAAAAGCTGGGCGATGGTTGTGGACCTGGACGAGCCACCCGTGCGATAGGTGAGATTCATTCTCGCCCTGACGGGTTGCGTCTGTCATCGGTCGATAAGGACTAACGGCCCTCAGGTGGGGACCAACGCCCCTGGGAGATAGCGGTCCCCGCCAGTAACGTACCGCTGAACCGACGGGATGTATCCGCCCCAGCGAAGGAGACGGCG",
"TCAGCACCATGACCGCCTGGCCACCAATCGCCCGTAACAAGCGGGACGTCCGCGACGACGCGTGCGCTAGCGCCGTGGCGGTGACAACGACCAGATATGGTCCGAGCACGCGGGCGAACCTCGTGTTCTGGCCTCGGCCAGTTGTGTAGAGCTCATCGCTGTCATCGAGCGATATCCGACCACTGATCCAAGTCGGGGGCTCTGGGGACCGAAGTCCCCGGGCTCGGAGCTATCGGACCTCACGATCACC"]


In [19]:
"""Exercise Break (1 point): Apply GreedyMotifSearch with pseudocounts to find motifs in the DosR 
dataset with k-mer length equal to 15 (click here to download).

"""

# set t equal to the number of strings in Dna and k equal to 15
k, t = 15, 10
# Call GreedyMotifSearchWithPseudocounts(Dna, k, t) and store the output in a variable called Motifs
Motifs = GreedyMotifSearchWithPseudocounts(Dna, k, t)
# Print the Motifs variable
print (Motifs)
# Print Score(Motifs)
print (Score(Motifs))

['GGACTTCAGGCCCTA', 'GGTCAAACGACCCTA', 'GGACGTAAGTCCCTA', 'GGATTACCGACCGCA', 'GGCCGAACGACCCTA', 'GGACCTTCGGCCCCA', 'GGACTTCTGTCCCTA', 'GGACTTTCGGCCCTG', 'GGACTAACGGCCCTC', 'GGACCGAAGTCCCCG']
80.0


You have now seen the power of pseudocounts illustrated on a small example. Running GreedyMotifSearch with pseudocounts to solve the Subtle Motif Problem returns a collection of 15-mers Motifs with Score(Motifs) = 41 and Consensus(Motifs) = "AAAAAtAgaGGGGtt". Thus, Laplace’s Rule of Succession has provided a significant improvement over the original GreedyMotifSearch, which returned the consensus string "gttAAAtAgaGatGtG" with Score(Motifs) = 58.

You may be satisfied with the performance of GreedyMotifSearch, but you should know by now that your authors are never satisfied. Can we design an even more accurate motif finding algorithm?